In [ ]:
import matplotlib.pyplot as plt
from pyqubo import Binary
from dwave.system.composites import EmbeddingComposite
from dwave.system.samplers import DWaveSampler
import numpy as np
import dimod

# 乱数シードを設定
seed_value = 20
np.random.seed(seed_value)

# 量子ビットをnつ用意する
players = 10
choice = 3
interaction_range = 1  # 前後n列のプレイヤーと相互作用を持つ
weight = 2

qbits = [Binary(f'q{i:03}') for i in range(players * choice)]

# 制約条件を記述する関数
def H_add(qbits, num):
    return sum(qbits) - num

# 制約条件を記述する
H = sum((H_add(qbits[i*choice:(i+1)*choice], 1)**2)*weight for i in range(players))

# 個人効用を追加
individual_utilities = []
for i in range(players * choice):
    individual_utility = 2 * np.random.rand() - 1
    individual_utilities.append(individual_utility)
    H += individual_utility * qbits[i]
    
# 相互作用を追加（前方のみ）
interaction_weights = []
for i in range(players):
    for j in range(1, interaction_range + 1):
        if i + j < players:
            for k in range(choice):
                for l in range(choice):
                    interaction_weight = 2 * np.random.rand() - 1  # -1から1の範囲で生成
                    interaction_weights.append((i, j, k, l, interaction_weight))
                    H += interaction_weight * qbits[i*choice + k] * qbits[(i+j)*choice + l]

# コンパイル
model = H.compile()
qubo, offset = model.to_qubo()
print('qubo\n{}'.format(qubo))

# D-waveの設定
sampler = EmbeddingComposite(DWaveSampler(endpoint='https://cloud.dwavesys.com/sapi',
                                          token='yourtoken',
                                          solver='Advantage_system6.4'))

num_reads_list = [100]
colors = ['blue', 'green', 'purple', 'orange']
labels = [f'num_reads={num_reads}' for num_reads in num_reads_list]

plt.figure(figsize=(20, 12))

total_occurrences = 0
total_ok_occurrences = 0

for num_reads, color, label in zip(num_reads_list, colors, labels):
    # サンプリングしまくる
    response = sampler.sample_qubo(qubo, num_reads=num_reads)

    # レスポンスの確認
    energies_ok = []
    occurrences_ok = []
    energies_ng = []
    occurrences_ng = []

    for (sample, energy, occurrence, _) in response.data():
        print('Sample:{} Energy:{} Occurrence:{}'.format(list(sample.values()), energy, occurrence))

        # 制約条件の検証
        valid = True
        for i in range(players):
            player_choices = sum(sample[f'q{j:03}'] for j in range(i * choice, (i + 1) * choice))
            if player_choices != 1:
                valid = False
                break

        if valid:
            print("OK")
            energies_ok.append(energy)
            occurrences_ok.append(occurrence / num_reads)  # 回数の割合に変更
            total_ok_occurrences += occurrence
        else:
            print("NG")
            energies_ng.append(energy)
            occurrences_ng.append(occurrence / num_reads)  # 回数の割合に変更

        total_occurrences += occurrence

    # グラフを描画
    plt.plot(energies_ok, occurrences_ok, color=color, linestyle='-', label=f'OK ({label})')
    plt.scatter(energies_ok, occurrences_ok, c=color, marker='o')  # OKの点を追加
    plt.scatter(energies_ng, occurrences_ng, c=color, marker='x', alpha=0.5, label=f'NG ({label})')  # NGの点を透明に

# OKの割合を計算して表示
ok_ratio = total_ok_occurrences / total_occurrences
print(f'OK ratio: {ok_ratio:.2%}')

# 量子ビット数が25を超える場合は最適解を求めない
if players * choice >= 25:
    print("The number of qubits exceeds the limit. Execution stopped.")
else:
    # dimodを使用して最適解を算出
    solver = dimod.ExactSolver()
    response_dimod = solver.sample_qubo(qubo)

    # 最適解のエネルギーを取得
    optimal_sample = response_dimod.first.sample
    optimal_energy = response_dimod.first.energy
    print('Optimal sample:', optimal_sample)
    print('Optimal energy:', optimal_energy)

    # 最適解をプロット
    plt.scatter([optimal_energy], [0], color='red', zorder=5, label='Optimal solution (dimod)')
    plt.axvline(x=optimal_energy, color='red', linestyle='--', label='Optimal energy')

plt.xlabel('Energy')
plt.ylabel('Occurrence Ratio')
plt.title('Energy vs Occurrence Ratio')
plt.legend()
plt.grid(True)

# ファイル名を設定して保存
file_name = f'result_combined_{min(num_reads_list)}_{max(num_reads_list)}_players{players}_choice{choice}_interaction{interaction_range}_weight{weight}_seed{seed_value}_{ok_ratio}.png'
plt.savefig(file_name)
plt.show()

In [ ]:
from collections import Counter
import dimod
import matplotlib.pyplot as plt
from itertools import combinations

# n個の解を対象とする
n = 10  # 任意に指定
sorted_samples = sorted(response.data(), key=lambda x: x[1])  # エネルギーの低い順にソート

# 近傍解を生成する関数
def generate_neighbors(sample, diff_count):
    neighbors = []
    for indices in combinations(range(len(sample)), diff_count):
        neighbor = sample.copy()
        for i in indices:
            neighbor[i] = 1 - neighbor[i]
        neighbors.append(neighbor)
    return neighbors

# 差異2の近傍解の最小エネルギーとの差を求める
energy_differences = []
for num in range(n):
    target_sample = sorted_samples[num][0]
    target_energy = sorted_samples[num][1]
    
    # 差異2の近傍解を生成
    neighbors = generate_neighbors(list(target_sample.values()), 2)
    
    # 近傍解のエネルギーを計算 (modelを使用せずに)
    neighbor_energies = []
    for neighbor in neighbors:
        neighbor_qubo = {f'q{i:03}': neighbor[i] for i in range(len(neighbor))}
        neighbor_energy = sum(qubo[(key1, key2)] * neighbor_qubo[key1] * neighbor_qubo[key2] for key1, key2 in qubo)
        neighbor_energies.append(neighbor_energy)
    
    # 最小エネルギーを取得
    min_neighbor_energy = min(neighbor_energies)
    
    # エネルギーの差を計算
    energy_difference = target_energy - min_neighbor_energy
    energy_differences.append(energy_difference)

# グラフを作成
plt.figure(figsize=(12, 8))
plt.bar(range(1, n+1), energy_differences, color='blue')
plt.xlabel('Rank of solution')
plt.ylabel('Energy difference')
plt.title('Energy difference between solution and its nearest neighbor with 2 differing qubits')
plt.grid(True)
plt.show()

In [ ]:
from collections import Counter
import dimod
import matplotlib.pyplot as plt
from itertools import combinations

# n番目に出現回数が多い解を取得
n = 3  # 例として3番目に出現回数が多い解を使用
sorted_samples = sorted(response.data(), key=lambda x: x[2], reverse=True)
nth_sample = sorted_samples[n-1][0]
nth_energy = sorted_samples[n-1][1]
print(f'{n}th sample:', nth_sample)
print(f'{n}th energy:', nth_energy)

# 近傍解を生成する関数
def generate_neighbors(sample, m):
    neighbors = {diff_count: [] for diff_count in range(2, m+1, 2)}
    for diff_count in range(2, m+1, 2):
        for indices in combinations(range(len(sample)), diff_count):
            neighbor = sample.copy()
            for i in indices:
                neighbor[i] = 1 - neighbor[i]
            neighbors[diff_count].append(neighbor)
    return neighbors

# 近傍解を生成
m = 6  # 例として差異が2からmまでの偶数の近傍解を使用
neighbors = generate_neighbors(list(nth_sample.values()), m)

# 近傍解のエネルギーを計算 (modelを使用せずに)
neighbor_energies_by_diff_count = {diff_count: [] for diff_count in range(2, m+1, 2)}
for diff_count, neighbor_list in neighbors.items():
    for neighbor in neighbor_list:
        neighbor_qubo = {f'q{i:03}': neighbor[i] for i in range(len(neighbor))}
        neighbor_energy = sum(qubo[(key1, key2)] * neighbor_qubo[key1] * neighbor_qubo[key2] for key1, key2 in qubo)
        neighbor_energies_by_diff_count[diff_count].append(neighbor_energy)

# グラフを作成
plt.figure(figsize=(12, 8))
boxplot_data = [energies for diff_count, energies in sorted(neighbor_energies_by_diff_count.items())]
labels = [f'Diff {diff_count}' for diff_count in sorted(neighbor_energies_by_diff_count.keys())]
plt.boxplot(boxplot_data, labels=labels)

# 3番目に出現回数が多い解のエネルギーをプロット
plt.scatter([0], [nth_energy], color='red', zorder=5, label='Nth Sample Energy')
plt.axhline(y=nth_energy, color='red', linestyle='--', label='Nth Sample Energy')

plt.xlabel('Difference Count')
plt.ylabel('Energy')
plt.title(f'Energy Comparison of Nth Sample and Its Neighbors (m={m})')
plt.legend()
plt.grid(True)
plt.show()